<a href="https://colab.research.google.com/github/MPaulignan/Projet_Backtest/blob/main/Backtest_Site.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# BACKTEST_WRAPPER
# ---------------------------
# Cette cellule contient les fonctions principales du backtest
# ---------------------------

import pandas as pd
import numpy as np
from datetime import time as dtime

# ---------------------------
# Configuration
# ---------------------------

# Paramètres risque
gfloatTPCoef: float = 1.3
gintSLQuotidienMax: int = 3
gfloatBalance: float = 10000
gfloatRisqueParTrade: float = 0.01

# ---------------------------
# calculate_ema_series
#     Calcule une EMA sur un historique long (au moins 5 fois la valeur de l'EMA)
# df [pandas.dataframe]
#     Historique des cotations sur lequel est calculé l'EMA. Si trop court, la fonction indique une erreur
# size [integer]
#     Taille de l'EMA calculée, les valeurs possibles sont 50 et 200
# ---------------------------
def calculate_ema_series(df: pd.DataFrame, size: int = 50) -> pd.Series:
    return df["close"].ewm(span=size, adjust=False).mean()

# ---------------------------
# detect_signal
#     compare les EMA sur un tableau de cotations pour identifier quand un signal est présent
# df [pandas.dataframe]
#     Historique des cotations
# ema_fast [integer]
#     EMA rapide à l'origine des signaux
# ema_slow [integer]
#     EMA lente pour indiquer la tendance
# ---------------------------
def detect_signal(df: pd.DataFrame, ema_fast: int = 50, ema_slow: int = 200) -> str:
    # Calcul des deux EMA sur les cotations fournies
    lfloatEMAFastVal = calculate_ema_series(df, ema_fast).iloc[-1]
    lfloatEMASlowVal = calculate_ema_series(df, ema_slow).iloc[-1]

    # Comparaison des EMA et des cotations pour identifier le signal
    if (lfloatEMAFastVal > lfloatEMASlowVal
        and df["open"].iloc[-1] < lfloatEMAFastVal
        and df["close"].iloc[-1] > lfloatEMAFastVal):
        return "bullish"

    elif (lfloatEMAFastVal < lfloatEMASlowVal
          and df["open"].iloc[-1] > lfloatEMAFastVal
          and df["close"].iloc[-1] < lfloatEMAFastVal):
        return "bearish"

    return "none"

# ---------------------------
# open_position
#     ouvre une nouvelle position
# trades [pandas.DataFrame]
#     Journal des positions prises
# time [datetime]
#     Date et heure à laquelle la position est prise
# price [float]
#     Prix au moment de l'entrée en position
# signal [string]
#     Signal d'entrée en position: "bullish", "bearish" ou "else"
# df [pandas.Dataframe]
#     Tableau contenant l'historique des cotations sur lequel le backtest est réalisé
# symbole [dictionary]
#     Caractéristiques du symbole sur lequel le backtest est réalisé
# ---------------------------
def open_position(trades: pd.DataFrame, time, price: float, signal: str, df: pd.DataFrame, symbole: dict) -> pd.DataFrame:
    # Vérifie qu'on est dans les bonnes heures
    if not gdtmDebutSession <= time.time() <= gdtmFinSession:
        return trades

    # Vérifie qu'il n'y a pas de position déjà ouverte
    if trades.shape[0] > 0 and trades["status"].iloc[-1] == "open":
        return trades

    # Vérifie que le nombre de SL max quotidien n'a pas été dépassé
    lintSLCount: int = 0
    lintSLCount = 0
    for _, pos in trades.iterrows():
        if pd.notna(pos["time_open"]) and time.date() == pos["time_open"].date() and pos["status"] == "closed_sl":
            lintSLCount += 1

    if lintSLCount >= gintSLQuotidienMax :
        return trades

    # Calcul de la mise
    lfloatMise: float = gfloatBalance * gfloatRisqueParTrade

    if signal == "bullish":

        # Calcul du prix du SL avec bornage sur le risque max et la distance SL plancher
        lfloatPrixSL: float = min(price - symbole["DistanceSLMin"],
                                  max(df["low"].iloc[-11:-1].min(), price - lfloatMise))

        # Calcul de la distance du SL
        lfloatDistanceSL: float = abs(price - lfloatPrixSL)

        # Calcul du nombre de ticks
        lintTicksCount: int = lfloatDistanceSL/symbole["TailleTick"]

        # Calcul de la perte par lot, ci elle est invalide, la fonction retourne une valeur par défaut
        lfloatPerteParLot: float = lintTicksCount * symbole["ValeurTick"] * symbole["TailleContrat"]
        if lfloatPerteParLot <= 0:
            return symbole["VolumeMin"]

        # lot correspondant au risque max de 5 €
        lfloatLotBrut: float = lfloatMise / lfloatPerteParLot

        # Calcul du lot max en fonction des paramètres globaux et du symbole pour limiter le risque
        lfloatVolumeMax: float = (
            lfloatMise/(symbole["DistanceSLMin"] / symbole["TailleTick"] * symbole["ValeurTick"]) * symbole["TailleContrat"])

        # Ajustement du volume en fonction du lot min du broker et du lot max calculé précédemment
        lfloatVolume: float = max(symbole["VolumeMin"], min(lfloatVolumeMax, lfloatLotBrut))

        ldictpos: dict = {
            "time_open": time,
            "price_open": price,
            "signal": signal,
            "position": "long",
            "sl": lfloatPrixSL,
            "tp": price + lfloatDistanceSL * gfloatTPCoef,
            "mise": lfloatPerteParLot * lfloatVolume,
            "status": "open",
            "price_close": np.nan,
            "time_close": np.nan,
            "profit": np.nan,
            "balance": np.nan
        }
    elif signal == "bearish":
        # Calcul du prix du SL avec bornage sur le risque max et la distance SL plancher
        lfloatPrixSL: float = max(price + symbole["DistanceSLMin"],
                                  min(df["high"].iloc[-11:-1].max(), price + lfloatMise))

        # Calcul de la distance du SL
        lfloatDistanceSL: float = abs(lfloatPrixSL - price)

        # Calcul du nombre de ticks
        lintTicksCount: int = lfloatDistanceSL/symbole["TailleTick"]

        # Calcul de la perte par lot, ci elle est invalide, la fonction retourne une valeur par défaut
        lfloatPerteParLot: float = lintTicksCount * symbole["ValeurTick"] * symbole["TailleContrat"]
        if lfloatPerteParLot <= 0:
            return symbole["VolumeMin"]

        # lot correspondant au risque max de 5 €
        lfloatLotBrut: float = lfloatMise / lfloatPerteParLot

        # Calcul du lot max en fonction des paramètres globaux et du symbole pour limiter le risque
        lfloatVolumeMax: float = (
            lfloatMise/(symbole["DistanceSLMin"] / symbole["TailleTick"] * symbole["ValeurTick"]) * symbole["TailleContrat"])

        # Ajustement du volume en fonction du lot min du broker et du lot max calculé précédemment
        lfloatVolume: float = max(symbole["VolumeMin"], min(lfloatVolumeMax, lfloatLotBrut))

        ldictpos: dict = {
            "time_open": time,
            "price_open": price,
            "signal": signal,
            "position": "short",
            "sl": lfloatPrixSL,
            "tp": price - lfloatDistanceSL * gfloatTPCoef,
            "mise": lfloatVolume * lfloatPerteParLot,
            "status": "open",
            "price_close": np.nan,
            "time_close": np.nan,
            "profit": np.nan,
            "balance": np.nan
        }
    else:
        return trades
    trades = pd.concat([trades, pd.DataFrame([ldictpos])], ignore_index=True)
    return trades

# ---------------------------
# update_positions
#     Met à jour de la dernière position selon la bougie actuelle si la position est ouverte
# trades [pandas.DataFrame]
#     Tableau récapitulatif des positions prises
# current_candle [pandas.Series]
#     Dernière ligne du tableau sur lequel le backtest est réalisé
# ---------------------------
def update_positions(trades: pd.DataFrame, current_candle: pd.Series) -> pd.DataFrame:
    # Vérifie qu'il y ait bien une position ouverte
    if trades.shape[0] == 0:
        return trades
    elif trades["status"].iloc[-1] != "open":
        return trades

    global gfloatBalance

    # Récupère les infos utiles concernant la dernière bougie
    lfloatLow: float = current_candle["low"]
    lfloatHigh: float = current_candle["high"]
    ldtmTime = current_candle["UTC"]

    # Passe en revue les cas de figure pouvant amener un résultat
    if trades["position"].iloc[-1] == "long":
        if lfloatLow <= trades["sl"].iloc[-1]:
            trades.loc[trades.index[-1], "status"] = "closed_sl"
            trades.loc[trades.index[-1], "price_close"] = trades["sl"].iloc[-1]
            trades.loc[trades.index[-1], "time_close"] = ldtmTime
            trades.loc[trades.index[-1], "profit"] = - trades["mise"].iloc[-1]

        elif lfloatHigh >= trades["tp"].iloc[-1]:
            trades.loc[trades.index[-1], "status"] = "closed_tp"
            trades.loc[trades.index[-1], "price_close"] = trades["tp"].iloc[-1]
            trades.loc[trades.index[-1], "time_close"] = ldtmTime
            trades.loc[trades.index[-1], "profit"] = trades["mise"].iloc[-1] * gfloatTPCoef

    elif trades["position"].iloc[-1] == "short":
        if lfloatHigh >= trades["sl"].iloc[-1]:
            trades.loc[trades.index[-1], "status"] = "closed_sl"
            trades.loc[trades.index[-1], "price_close"] = trades["sl"].iloc[-1]
            trades.loc[trades.index[-1], "time_close"] = ldtmTime
            trades.loc[trades.index[-1], "profit"] = - trades["mise"].iloc[-1]

        elif lfloatLow <= trades["tp"].iloc[-1]:
            trades.loc[trades.index[-1], "status"] = "closed_tp"
            trades.loc[trades.index[-1], "price_close"] = trades["tp"].iloc[-1]
            trades.loc[trades.index[-1], "time_close"] = ldtmTime
            trades.loc[trades.index[-1], "profit"] = trades["mise"].iloc[-1] * gfloatTPCoef

    # Enregistre l'impact du résultat sur la balance globale
    if trades["status"].iloc[-1] in ["closed_tp", "closed_sl"]:
        gfloatBalance += trades["profit"].iloc[-1]
        trades.loc[trades.index[-1], "balance"] = gfloatBalance

    return trades

# ---------------------------
# analyze_performance
#     Analyse les performances du backtest
# trades [pandas.dataframe]
#     Journal contenant les trades à analyser
# ---------------------------
def analyze_performance(trades):
    closed = trades[trades["status"].isin(["closed_sl", "closed_tp"])]
    net = closed["profit"].sum()
    winrate = (closed["status"] == "closed_tp").mean() * 100
    drawdown = closed["balance"].cummax() - closed["balance"]
    max_dd = drawdown.max()

    # --- Calcul du nombre max de SL consécutifs
    lintSlCountMax: int = 0
    lintSlCount: int = 0
    for status in closed["status"]:
        if status == "closed_sl":
            lintSlCount += 1
            lintSlCountMax = max(lintSlCountMax, lintSlCount)
        else:
            lintSlCount = 0

    return pd.Series({"net": net,
                      "winrate": winrate,
                      "drawdown": max_dd,
                      "nb_trades": len(closed),
                      "marge_erreur": 1.96*np.sqrt(0.5*(1-0.5)/len(closed))*100,
                      "Max_SL_Serie": lintSlCountMax})

# ---------------------------
# run_backtest
#     Fonction principale qui contrôle l'exécution du backtest
# ---------------------------
def run_backtest(Actif: dict, fileCotations, EMAFast: int = 50, EMASlow: int = 200):
    ldfM5: pd.DataFrame = pd.read_excel(fileCotations)
    ldfM5["UTC"] = pd.to_datetime(ldfM5["UTC"])

    ldfTrades: pd.DataFrame = pd.DataFrame(columns=("time_open", "price_open", "signal",
                                                    "position", "sl", "tp", "mise",
                                                    "status", "price_close", "time_close",
                                                    "profit", "balance"))

    for i in range(max(EMAFast, EMASlow), len(ldfM5)):
        ldfSubset = ldfM5.iloc[:i+1]
        ldsCurrent = ldfM5.iloc[i]

        # Clôturer les positions ouvertes
        ldfTrades = update_positions(ldfTrades, ldsCurrent)

        # Vérifie si conditions d'ouverture
        lstrSignal: str = detect_signal(ldfSubset, EMAFast, EMASlow)
        if lstrSignal in ["bullish", "bearish"]:
            ldfTrades = open_position(ldfTrades, ldsCurrent["UTC"], ldsCurrent["close"], lstrSignal, ldfSubset, Actif)

    # Calcule les statistiques du backtest
    stats = analyze_performance(ldfTrades)

    print(f"✅ Backtest EMA{EMAFast}/{EMASlow} terminé")
    print(f"sur: {Actif["Symbole"]}, de {gdtmDebutSession.hour}h à {gdtmFinSession.hour}h")
    print(stats)
    return (ldfTrades)

In [3]:
# BACKTESTS XAU/USD
# ---------------------------
# Configuration du backtest
#     Variables principales à régler pour lancer un backtest
# Cette cellule est dédiée aux tests sur "XAUUSD"
# ---------------------------

# Devises testée
gdictActif: dict = {"Symbole": "XAUUSD",
            "TailleTick": 0.01,
            "ValeurTick": 0.01,
            "TailleContrat": 100,
            "VolumeMin": 0.01,
            "VolumeMax": 100,
            "DistanceSLMin": 0.5}

# Paramètres stratégie
gintEMAFast: int = 50
gintEMASlow: int = 200
gdtmDebutSession = dtime(6, 0)
gdtmFinSession = dtime(10, 0)

# Fichier à charger
gfileM5 = r"https://github.com/MPaulignan/Projet_Backtest/blob/1215aa6a6b43b1d2c9785a03b34189263cc22887/EURUSD_M5.xlsx"
# ---------------------------
# Exécution du backtest
# ---------------------------

ldfJournal: pd.DataFrame = run_backtest(gdictActif, gfileM5, gintEMAFast, gintEMASlow)

ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [ ]:
# ENREGISTREMENT JOURNAL
# ---------------------------
# Cette cellule permet l'enregistrement d'un journal jugé intéressant
# ---------------------------

output_path = fr"https://github.com/MPaulignan/Projet_Backtest/Journal_EMA{gintEMAFast}{gintEMASlow}{gdictActif["Symbole"]}.xlsx"
ldfJournal.to_excel(output_path, index=False)
print("✅ Le journal a bien été enregistré!")